# CoDA (entropy discovery) — near/far demo

Uses your sequences:
```
near = [1,1,1,1,1,1, 2,2,2,2, 1,1,1, 4,6, 1,1,1, 5,5, 1,1, 7, 0,0,0]
far  = [1,1,1,1,1,1, 3,3,3,3, 1,1,1, 4,4, 1,1,1, 5,6, 1,1, 7, 0,0,0]
```

Entropy discovery makes the aliased transitions at some states look stochastic across episodes.


In [ ]:
!pip -q install numpy networkx matplotlib

In [ ]:
from coda_entropy_zipper_core import CoDAEntropyZipperAgent, CoDAEntropyConfig, EligibilityTraceConfig, DiscoveryConfig
from coda_envs import NearFarSeqEnv
from coda_graph_viz import save_snapshot_sequence, plot_snapshot
import os


In [ ]:
env = NearFarSeqEnv(seed=0, p_near=0.5)

cfg = CoDAEntropyConfig(
    et=EligibilityTraceConfig(gamma=0.98, lam=0.95, mode='first_visit'),
    disc=DiscoveryConfig(mode='entropy', min_sa_count=10.0, entropy_threshold=0.40),
    n_threshold=6.0,
    theta_split=0.78,
    theta_merge=0.40,
    zipper_sweep_only_when_new_cue=True,
)
agent = CoDAEntropyZipperAgent(cfg)


In [ ]:
snapshots = []
snap_every = 50
n_episodes = 600

for ep in range(n_episodes):
    obs_seq, act_seq, lab = env.sample_episode()
    diag = agent.process_episode(obs_seq, act_seq)
    if ep % snap_every == 0:
        snapshots.append(agent.snapshot(title=f'Graph at iteration {len(snapshots)} (ep={ep})'))

final_snap = agent.snapshot(title=f'Graph at iteration final (ep={n_episodes})')
snapshots.append(final_snap)

diag, (len(agent.G.nodes), len(agent.salient))


In [ ]:
plot_snapshot(final_snap, layout='spring', node_label='sid_obs', title=final_snap.title);


In [ ]:
out_dir = 'nearfar_entropy_steps'
paths = save_snapshot_sequence(snapshots, out_dir, prefix='nearfar_iter', layout='spring', node_label='sid_obs')
paths[:3], '...'
